In [3]:
# !pip install geocoder

In [38]:
import geocoder
coordinates = geocoder.arcgis(f'台北市信義區信義路五段7號').latlng
coordinates


In [1]:
import geocoder
from shapely.geometry import Point


address = "10684台北市大安區東豐街30號"
# 輸入地址轉經緯度
coordinates = geocoder.arcgis(address).latlng

# 給定經度、緯度
longitude = coordinates[1]
latitude = coordinates[0]


# 創建中心點
center_point = Point(longitude, latitude)

In [2]:
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin


# 建立中心點gdf
center_gdf = gpd.GeoDataFrame(
                    geometry=[center_point],
                    crs = "epsg:4326" ) 
# 設置原始數據的 CRS 為 WGS 84

# 載入資料集
# 指定存放CSV文件的資料夾路徑
folder_path = r'./Dataset/'  # 確保使用r來避免轉義字符的問題


# 雙北飲料店
drink_df = gpd.read_file(f'{folder_path}drink.csv')
drink_gdf = gpd.GeoDataFrame(
                drink_df, 
                geometry=[Point(xy) for xy in zip(drink_df.longitude, drink_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北火車
train_df = gpd.read_file(f'{folder_path}train.csv')
train_gdf = gpd.GeoDataFrame(
                train_df, 
                geometry=[Point(xy) for xy in zip(train_df.longitude, train_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北公車站
bus_df = gpd.read_file(f'{folder_path}bus.csv')
bus_gdf = gpd.GeoDataFrame(
                bus_df, 
                geometry=[Point(xy) for xy in zip(bus_df.longitude, bus_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北公園
park_df = gpd.read_file(f'{folder_path}park.csv')
park_gdf = gpd.GeoDataFrame(
                park_df, 
                geometry=[Point(xy) for xy in zip(park_df.longitude, park_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北夜市
night_market_df = gpd.read_file(f'{folder_path}night_market.csv')
night_market_gdf = gpd.GeoDataFrame(
                night_market_df, 
                geometry=[Point(xy) for xy in zip(night_market_df.longitude, night_market_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北捷運
mrt_df = gpd.read_file(f'{folder_path}mrt.csv')
mrt_gdf = gpd.GeoDataFrame(
                mrt_df, 
                geometry=[Point(xy) for xy in zip(mrt_df.longitude, mrt_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北學校
sports_facilities_df = gpd.read_file(f'{folder_path}sports_facilities.csv')
sports_facilities_gdf = gpd.GeoDataFrame(
                sports_facilities_df, 
                geometry=[Point(xy) for xy in zip(sports_facilities_df.longitude, sports_facilities_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北電影院
movie_theater_df = gpd.read_file(f'{folder_path}movie_theater.csv')
movie_theater_gdf = gpd.GeoDataFrame(
                movie_theater_df, 
                geometry=[Point(xy) for xy in zip(movie_theater_df.longitude, movie_theater_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北學校
school_df = gpd.read_file(f'{folder_path}school.csv')
school_gdf = gpd.GeoDataFrame(
                school_df, 
                geometry=[Point(xy) for xy in zip(school_df.longitude, school_df.latitude)],
                crs = "epsg:4326" 
            )


# 雙北醫院
hospital_df = gpd.read_file(f'{folder_path}hospital.csv')
hospital_gdf = gpd.GeoDataFrame(
                    hospital_df,
                    geometry=[Point(xy) for xy in zip(hospital_df.longitude, hospital_df.latitude)],
                    crs = "epsg:4326" 
                )

# school_gdf

In [3]:
# 轉換坐標系統到 Web Mercator (epsg:3826(TWD97 121分帶)) 以計算以公尺為單位距離
center_gdf = center_gdf.to_crs(epsg=3826)
school_gdf = school_gdf.to_crs(epsg=3826)
drink_gdf = drink_gdf.to_crs(epsg=3826)
train_gdf = train_gdf.to_crs(epsg=3826)
bus_gdf = bus_gdf.to_crs(epsg=3826)
park_gdf = park_gdf.to_crs(epsg=3826)
night_market_gdf = night_market_gdf.to_crs(epsg=3826)
sports_facilities_gdf = sports_facilities_gdf.to_crs(epsg=3826)
mrt_gdf = mrt_gdf.to_crs(epsg=3826)
movie_theater_gdf = movie_theater_gdf.to_crs(epsg=3826)
hospital_gdf = hospital_gdf.to_crs(epsg=3826)

In [4]:
# 建立中心點的id(唯一值)
center_gdf['shop_id'] = range(len(center_gdf))

# 建立緩衝區(輪廓) GeoDataFrame
buffer_gdf = gpd.GeoDataFrame(center_gdf[['shop_id', 'geometry']].copy())
buffer_gdf['geometry'] = buffer_gdf.geometry.buffer(1000, resolution=99) # 建立1公里緩衝區(輪廓)

buffer_gdf

,shop_id,geometry
0,0,"POLYGON ((306054.341 2769869.277, 306054.215 2..."


In [5]:
# 空間連接(join)
school_joined_gdf = sjoin(buffer_gdf, school_gdf, how="inner", predicate='contains')
drink_joined_gdf = sjoin(buffer_gdf, drink_gdf, how="inner", predicate='contains')
train_joined_gdf = sjoin(buffer_gdf, train_gdf, how="inner", predicate='contains')
bus_joined_gdf = sjoin(buffer_gdf, bus_gdf, how="inner", predicate='contains')
park_joined_gdf = sjoin(buffer_gdf, park_gdf, how="inner", predicate='contains')
night_market_joined_gdf = sjoin(buffer_gdf, night_market_gdf, how="inner", predicate='contains')
sports_facilities_joined_gdf = sjoin(buffer_gdf, sports_facilities_gdf, how="inner", predicate='contains')
mrt_joined_gdf = sjoin(buffer_gdf, mrt_gdf, how="inner", predicate='contains')
movie_theater_joined_gdf = sjoin(buffer_gdf, movie_theater_gdf, how="inner", predicate='contains')
hospital_joined_gdf = sjoin(buffer_gdf, hospital_gdf, how="inner", predicate='contains')

# 執行統計每個緩沖區內的設施數量
school_counts = school_joined_gdf.groupby('shop_id').size()
drink_counts = drink_joined_gdf.groupby('shop_id').size()
train_counts = train_joined_gdf.groupby('shop_id').size()
bus_counts = bus_joined_gdf.groupby('shop_id').size()
park_counts = park_joined_gdf.groupby('shop_id').size()
night_market_counts = night_market_joined_gdf.groupby('shop_id').size()
sports_facilities_counts = sports_facilities_joined_gdf.groupby('shop_id').size()
mrt_counts = mrt_joined_gdf.groupby('shop_id').size()
movie_theater_counts = movie_theater_joined_gdf.groupby('shop_id').size()
hospital_counts = hospital_joined_gdf.groupby('shop_id').size()


# 需要將計數結果與原始的多邊形 GeoDataFrame 進行合併
# 為了確保即使是數值為 0 的多邊形也能被統計
buffer_gdf['school_counts'] = buffer_gdf.index.map(school_counts).fillna(0)
buffer_gdf['drink_counts'] = buffer_gdf.index.map(drink_counts).fillna(0)
buffer_gdf['train_counts'] = buffer_gdf.index.map(train_counts).fillna(0)
buffer_gdf['bus_counts'] = buffer_gdf.index.map(bus_counts).fillna(0)
buffer_gdf['park_counts'] = buffer_gdf.index.map(park_counts).fillna(0)
buffer_gdf['night_market_counts'] = buffer_gdf.index.map(night_market_counts).fillna(0)
buffer_gdf['sports_facilities_counts'] = buffer_gdf.index.map(sports_facilities_counts).fillna(0)
buffer_gdf['mrt_counts'] = buffer_gdf.index.map(mrt_counts).fillna(0)
buffer_gdf['movie_theater_counts'] = buffer_gdf.index.map(movie_theater_counts).fillna(0)
buffer_gdf['hospital_counts'] = buffer_gdf.index.map(hospital_counts).fillna(0)


school_counts = int(buffer_gdf['school_counts'].values[0])
drink_counts = int(buffer_gdf['drink_counts'].values[0])
train_counts = int(buffer_gdf['train_counts'].values[0])
bus_counts = int(buffer_gdf['bus_counts'].values[0])
park_counts = int(buffer_gdf['park_counts'].values[0])
night_market_counts = int(buffer_gdf['night_market_counts'].values[0])
sports_facilities_counts = int(buffer_gdf['sports_facilities_counts'].values[0])
mrt_counts = int(buffer_gdf['mrt_counts'].values[0])
movie_theater_counts = int(buffer_gdf['movie_theater_counts'].values[0])
hospital_counts = int(buffer_gdf['hospital_counts'].values[0])

print(
    f'學校數量 : {school_counts}''\n'
    f'飲料店數量 : {drink_counts}''\n'
    f'火車站數量 : {train_counts}''\n'
    f'公車站數量 : {bus_counts}''\n'
    f'公園數量 : {park_counts}''\n'
    f'夜市數量 : {night_market_counts}''\n'
    f'運動設施數量 : {sports_facilities_counts}''\n'
    f'捷運站數量 : {mrt_counts}''\n'
    f'電影院數量 : {movie_theater_counts}''\n'
    f'醫院數量 : {hospital_counts}''\n'
)


學校數量 : 12
飲料店數量 : 144
火車站數量 : 0
公車站數量 : 37
公園數量 : 24
夜市數量 : 0
運動設施數量 : 24
捷運站數量 : 4
電影院數量 : 1
醫院數量 : 4



In [30]:
center_gdf['shop_id'] = range(len(center_gdf))

0    0
Name: shop_id, dtype: int64

In [31]:
import geopandas as gpd
from shapely.geometry import Point

# 假设我们有一些中心点，这里我们随机创建一些示例点
points = [Point(1, 2), Point(2, 3), Point(3, 4)]
center_gdf = gpd.GeoDataFrame(geometry=points)

# 为每个点分配一个唯一的 shop_id
center_gdf['shop_id'] = range(len(center_gdf))

# 显示结果
print(center_gdf)

                  geometry  shop_id
0  POINT (1.00000 2.00000)        0
1  POINT (2.00000 3.00000)        1
2  POINT (3.00000 4.00000)        2


In [10]:
import os
os.path.abspath(os.path.dirname(__file__))

NameError: name '__file__' is not defined

In [9]:
import os
print(os.getcwd())


C:\Users\student\python_web_scraping-master\final_project


In [11]:
import os
current_dir = os.getcwd()
print(current_dir)


C:\Users\student\python_web_scraping-master\final_project
